In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
import featuretools as ft
from feature_selector import FeatureSelector
import time
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')

In [3]:
start = time.time()

# Read meta data
meta = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/input/meta_open.csv', 
                   index_col='uid', parse_dates=['dataend','datastart'], dayfirst=True)#The data will be messed up withou specifying dayfirst


# Read energy data
temporal = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/input/temp_open_utc_complete.csv', 
                   index_col='timestamp', parse_dates=True)#.tz_localize('utc')

def loopModels_and_Metrics(ml_Models_names, ml_Models, weatherPoints, cor_threshold, 
                           buildingNames, agg_primitives, trans_primitives, varianceThreshold, SelectPercentile_num,
                           SelectPercentile_func, RFE_step):  
    print('\n\n' + ml_Models_names + '\n_____________')
    buildingindex = 0
    for single_building in buildingNames:
        buildingindex+=1
        print('Modelling:' + single_building)
        
        # Read energy data for each given buildingname
        single_timezone = meta.T[single_building].timezone
        startdate = meta.T[single_building].datastart
        enddate = meta.T[single_building].dataend
        single_building_energy = temporal[single_building].tz_convert(single_timezone).truncate(before = startdate, 
                                                            after = enddate)#.fillna(method='bfill').fillna(method='ffill')
                                                            # single_building_energy, some missing data


        # Get weather data for given building
        weatherfile_name = meta.T[single_building].newweatherfilename
        weather_data = pd.read_csv(os.path.join('/Users/t.wang/Desktop/Dissertation/Python/input/',
                                                weatherfile_name),index_col='timestamp', parse_dates=True, na_values='-9999')
        weather_data = weather_data.tz_localize(single_timezone, ambiguous = 'infer')
        weather_point_list=[]
        for point in weatherPoints:
            point_data = weather_data[[point]]
            weather_point_list.append(point_data)
            all_weather_point = pd.concat(weather_point_list,axis=1) #axis=1, rowwise concat
            all_weather_point = all_weather_point[~all_weather_point.index.duplicated()]#To get rid of duplicated index
            all_weather_point = all_weather_point.reindex(pd.DatetimeIndex(start = all_weather_point.index[0], 
                                                                           periods=len(single_building_energy), 
                                                                           freq='H')).fillna(method='ffill').fillna(method='bfill')
#             in some cases, there are more than 1 data in the same hour, creating more than 8760 points
#             to make them consistent, take the first index minuits, based on the number of energy data,
#             transform them into hourly data. Then we get the same number of energy data (mostly8760)
#             DatatimeIndex them, reindex then is able to match and select those hour with the minuites
#             same as first index, regulating the data to be consistent with number of energy points, get
#             rid of the repeated weather data in the same hour.
    
        # Get schedule data for given building
        schedule_name = meta.T[single_building].annualschedule
        schedule_data = pd.read_csv(os.path.join('/Users/t.wang/Desktop/Dissertation/Python/input/',
                                                schedule_name),index_col=0, header=None, parse_dates=True)
        schedule_data = schedule_data.tz_localize(single_timezone, ambiguous = 'infer')
        schedule_data.columns = ['seasonal']
        schedule_data = schedule_data.reindex(pd.DatetimeIndex(start = schedule_data.index[0], periods=len(single_building_energy), 
                                                               freq='H')).fillna(method='ffill').fillna(method='bfill')
#         same trick is applied to selecting schedule data


        
        features = pd.merge(pd.DataFrame(single_building_energy.index.tz_localize(None)), 
                    schedule_data.reset_index(drop=True), right_index=True, left_index=True)#remove the time zone information
                #Map the schedule, otherwise the TimeSplits will not be able to capture all schedules, resulting in inconsistency of traning/test feature dimensions
        features['seasonal_num'] = features.seasonal.map({'Break':0, 'Regular':1, 'Holiday':2, 'Summer':3})
        features = features.drop('seasonal', axis=1)
        features = pd.concat([features, all_weather_point.reset_index(drop=True)], axis=1) #.reset_index(drop=True) to get rid of the time index, otherwise two sets data will stratify
#                 features = features.fillna(method='ffill').fillna(method='bfill')
        # features = np.array(features)
        labels = single_building_energy.values
        '''FeatureTool'''
        es = ft.EntitySet(id = 'buildingFeatures') #create Entity set
        # create an entity from feature table, unique index is created
        es = es.entity_from_dataframe(entity_id='featureData', dataframe=features,
                      make_index=True, index='feature_id', time_index = 'timestamp')

        features_FE, feature_names = ft.dfs(entityset = es, target_entity = 'featureData', max_depth = 2
                        ,agg_primitives = agg_primitives,
                        trans_primitives = trans_primitives, verbose = True, n_jobs=1) #Not sure why n_jobs more than 1 is not working

        # one hot encoding for categorical data
        features_enc, feature_names_enc = ft.encode_features(features_FE, feature_names)
        # Replace infinity number arising after feature generation
        features_enc = features_enc.replace(np.inf, '9999')
        features_enc = features_enc.replace(-np.inf, '-9999')
        features_enc = features_enc.replace([np.nan,''],0)
#         print(features_enc)

        '''Feature Selection'''
#                 Filter methods - Remove collinear features - FeatureSelector
        y = labels
        X = features_enc
        fs = FeatureSelector(data = X, labels = y)
        fs.identify_collinear(correlation_threshold = cor_threshold, one_hot=False)
        X_collinear = fs.remove(methods = ['collinear'], keep_one_hot=False)
        
#         Filter methods - Remove features with low variance - SKlearn
        sel = VarianceThreshold(threshold=(varianceThreshold * (1 - varianceThreshold)))
        X_collinear_Variance = sel.fit_transform(X_collinear)
        X_collinear_Variance = pd.DataFrame(X_collinear_Variance)

#                 Filter methods - SelectPercentile based on mutual information - SKlearn   
        X_collinear_Variance_MI = SelectPercentile(score_func=SelectPercentile_func, percentile=SelectPercentile_num).fit_transform(X_collinear_Variance,y)
#         Extract one year data for later Wrapper methods comparison
        pd.DataFrame(X_collinear_Variance_MI).to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/One_year_data/'+ str(single_building)+'_One_year_features' + '.csv', index=False)
        pd.DataFrame(y).to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/One_year_data/'+ str(single_building)+'_One_year_target' + '.csv', index=False)
                                

    
ml_Models_lists = [['RandomForestRegressor_FE', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)]]
weatherPoints = ['TemperatureC', 'Humidity','Dew PointC','Sea Level PressurehPa', 
                 'Wind Direction','Conditions','WindDirDegrees']


chosen_buildings = [1,2,3,165,166,167,288,289,290,360,361,362,450,451,452]
#drop buildings with missing schedule
buildingNames = meta.dropna(subset=['annualschedule']).index[chosen_buildings]
    
agg_primitives = []
trans_primitives = ['weekday','hour','is_weekend','divide_numeric',
                               'and','multiply_numeric','divide_by_feature','absolute','week','subtract_numeric',
                               'percentile']
cor_threshold = 0.5
varianceThreshold = 0.8
SelectPercentile_num = 5
SelectPercentile_func = mutual_info_regression
RFE_step = 10

for elem in ml_Models_lists:
#     ml_Models_names = elem[0], ml_Models = elem[1], not sure why this gives warning 'no n_estimator'
    loopModels_and_Metrics(ml_Models_names = elem[0], ml_Models=elem[1],weatherPoints=weatherPoints,
                           buildingNames=buildingNames, cor_threshold = cor_threshold,
                           agg_primitives=agg_primitives, trans_primitives=trans_primitives,varianceThreshold=varianceThreshold,
                           SelectPercentile_num=SelectPercentile_num, SelectPercentile_func=SelectPercentile_func, RFE_step=RFE_step)

end = time.time()
elapsed = end - start 
print('Time per building after FE and FS:'+ time.strftime("%H:%M:%S", time.gmtime(elapsed)))


# all_weather_point
# schedule_data
# single_building_energy
# train_test_list
# X_train,y_train
# X_train.shape,y_train.shape
# X_test,y_test
# X_test.shape,y_test.shape
# buildingNames




RandomForestRegressor_FE
_____________
Modelling:Office_Abigail
Built 41 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks
15 features with a correlation magnitude greater than 0.50.

Data has not been one-hot encoded
Removed 15 features including one-hot features.
Modelling:Office_Al
Built 41 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks
15 features with a correlation magnitude greater than 0.50.

Data has not been one-hot encoded
Removed 15 features including one-hot features.
Modelling:Office_Alannah
Built 41 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks
15 features with a correlation magnitude greater than 0.50.

Data has not been one-hot encoded
Removed 15 features including one-hot features.
Modelling:PrimClass_Jamie
Built 41 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks
8 features

# Check csv with specified columns (practice, irrelevant to above)

In [35]:
# You can just read a single line using nrows=1 to get the cols and then re-read in the 
# full csv skipping the first col by slicing the column array from the first read

cols = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/RandomForestRegressor_metrics_cross_validation_1.csv', nrows=1).columns
df=pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/RandomForestRegressor_metrics_cross_validation_1.csv',usecols=cols[1:])
df.head()

,MAPE,NMBE,CVRSME,RSQUARED,trainedMonths_,testMonths_
0,11.571059,1.746917,16.036343,0.432561,[1 2 3],[4 5 6]
1,7.379363,5.686699,9.761787,0.592209,[1 2 3],[4 5 6]
2,6.512372,4.410711,9.115582,0.601550,[1 2 3],[4 5 6]
3,8.740142,-1.536381,10.919086,0.515439,[1 2 3],[4 5 6]
4,12.210618,-5.639330,14.834996,0.433792,[1 2 3],[4 5 6]
